In [3]:
import pandas as pd
 
#sExcelFile="http://basic.10jqka.com.cn/api/stock/export.php?export=main&type=report&code=000002"

#df = pd.read_excel(sExcelFile,sheet_name='Worksheet',header=None)

In [ ]:
browser.get("https://bot.sannysoft.com/") #检测验证浏览器是否selenuim受控等信息

In [123]:
###查找图片点击中间

import pyautogui
s = pyautogui.locateOnScreen('dcsj.png')
print( s)
pyautogui.moveTo(s[0]+s[2]/2, s[1]+s[3]/2, duration=0.25)
pyautogui.click()

Box(left=500, top=599, width=80, height=29)


In [3]:
####隐藏selenuim模式，防止被网站识别为机器人

from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options, executable_path='./chromedriver')
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
  "source": """
    Object.defineProperty(navigator, 'webdriver', {
      get: () => undefined
    })
  """
})
#driver.get('http://exercise.kingname.info')  #测试练习网页
driver.get('http://stockpage.10jqka.com.cn/000651/finance/#view')
driver.switch_to.frame(driver.find_element_by_id("dataifm"))#处理iframe下查不到元素问题
driver.find_elements_by_xpath('//*[@id="exportButton"]')[0].click()

#Todo 写一个函数下载 参数：代码，报表类型（主要指标，负债表，利润表，现流表），期间类型（年，季，最新）

In [ ]:
#coding=utf-8
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

#driver = webdriver.Chrome() #(executable_path="C:/chromedriver.exe")
#driver.get('https://www.baidu.com')

try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "dataifm"))
    )
    #element.click()
    driver.switch_to.frame(driver.find_element_by_id("dataifm"))#处理iframe下查不到元素问题
    driver.find_elements_by_xpath('//*[@id="exportButton"]')[0].click()
finally:
    time.sleep(10)
    driver.quit()	

In [162]:
####下载同花顺财务报表,主程序

import os
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium import webdriver
from selenium.webdriver.remote.command import Command


StockCode = '000002'   #默认打开
repPath   = r"d:\rep" #报表下载后保存目录

def get_drv():
    global driver ,StockCode
    options = webdriver.ChromeOptions()
    prefs = {"download.default_directory": repPath}
    options.add_experimental_option("prefs", prefs)  #修改下载默认路径,方便读写
    options.add_experimental_option("excludeSwitches", ["enable-automation"])  #以下隐藏selenium的机器人状态,防止同花顺识别出来.
    options.add_experimental_option('useAutomationExtension', False)    
    driver = webdriver.Chrome(options=options, executable_path='./chromedriver')
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
      "source": """
        Object.defineProperty(navigator, 'webdriver', {
          get: () => undefined
        })
      """
    })
    #driver.get('http://stockpage.10jqka.com.cn/000651/finance/#view')
    return driver 

def open_drv():
    global driver ,StockCode
    if driver :  #判断chrome 是不是已经打开好了同花顺网站
        try :
            if driver.current_url.find('10jqka.com.cn/' + StockCode ) == -1 :
                driver.get('http://stockpage.10jqka.com.cn/'+ StockCode +'/finance/')
                print(StockCode + ' is opened.')
                return True
            else :
                print('chrome is opened.')
                return True             
        except Exception as e :
            print(e,' 尝试再次启动chrome，稍等。')
        #else :
            driver  = get_drv()
    try :
        driver.get('http://stockpage.10jqka.com.cn/'+ StockCode +'/finance/')
    except  Exception as e :
            print(e)
            return False        
    return True
#driver.get('http://exercise.kingname.info')  #测试练习网页

def get_urls(code="000002",export='',type=""):
    #todo ： 实现与本地文件比对之后决定下载哪些，已经下载的最新报表不必重复下载
    urls = []
    global driver , StockCode 
    StockCode = code    
    if export != '' :
        urls.append( "http://basic.10jqka.com.cn/api/stock/export.php?export=" + export)
    else :
        urls.append( "http://basic.10jqka.com.cn/api/stock/export.php?export=main" )
        urls.append( "http://basic.10jqka.com.cn/api/stock/export.php?export=debt" )
        urls.append( "http://basic.10jqka.com.cn/api/stock/export.php?export=benefit" )
        urls.append( "http://basic.10jqka.com.cn/api/stock/export.php?export=cash" )
    
    if type == '' :
        my_type = 'report'
    else :
        my_type = type
    
    for i in range(len(urls)) :
        urls[i] = urls[i] + "&type=" + my_type + "&code=" + code 
    
    return urls

        
        
def get_rep(code="",export="",type=""):
    '''
    code   : A股6位代码
    exoprt : (main:主要指标,debet:负债表,benefit:利润表,cash:现金流量表)
    type   : (report:最新报告期,year:年报,simple:单季报)
    http://basic.10jqka.com.cn/api/stock/export.php?export=main&type=report&code=000002
    http://basic.10jqka.com.cn/api/stock/export.php?export=debt&type=year&code=000002
    http://basic.10jqka.com.cn/api/stock/export.php?export=benefit&type=simple&code=000002
    http://basic.10jqka.com.cn/api/stock/export.php?export=cash&type=report&code=000002    
    '''
    global driver , StockCode 
    #StockCode = code
    urls = get_urls(code,export,type)
    #print(StockCode)
    if open_drv() :
        pass
    else :
        print('未能打开chrome')
        return
    #url = "http://basic.10jqka.com.cn/api/stock/export.php?export=" + export +"&type=" + type + "&code=" + code 
    
    #driver.get('http://stockpage.10jqka.com.cn/000002/finance/#view')
    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "dataifm"))
        )
        print(driver.title) #打印当前网页标题
        #element.click()
        #driver.switch_to.frame(driver.find_element_by_id("dataifm"))#处理iframe下查不到元素问题
        #driver.find_elements_by_xpath('//*[@id="exportButton"]')[0].click()
        #driver.get('http://basic.10jqka.com.cn/api/stock/export.php?export=cash&type=report&code=600751')
        for url in urls :
            driver.get(url)  
            driver.back
            time.sleep(2)
    finally:
        #time.sleep(3)
        
        print(StockCode,' 财报下载到了',repPath)
        #os.startfile(repPath)  #去掉注释，打开下载目录
        #driver.quit()

def get_reps(codes="",export="",type=""):  #codes:'603885,300146,600048',逗号分隔实现多股下载
    for code in codes.split(','):
        get_rep(code,export,type)
        
        
        
#if 'driver' not in vars():
driver=get_drv()

get_reps('000002')


# todo 
# pd 分析，竞争对手对比，查找本地文件，没有就从网上下载

000002 is opened.
万科A(000002)资金流向_个股行情_同花顺财经
000002  财报下载到了 d:\rep


In [7]:
driver.get('http://stockpage.10jqka.com.cn/000651/finance/#view')

In [16]:
####打开文件夹
import os 
start_directory = r'd:\rep'
os.startfile(start_directory)

In [14]:
#   Deprecated chrome option is ignored: useAutomationExtension  #打开状态
#   Unable to evaluate script: disconnected: not connected       #关闭状态

driver.get_log('driver')[-1]['message'].find('Unable to evaluate script: disconnected: not connected')

-1

In [18]:
driver.get_log('driver')[-1]['message'].find('Unable to evaluate script: disconnected: not connected')

0

In [21]:
driver.quit()

In [24]:
driver.get_log('driver')[-1]

{'level': 'WARNING',
 'message': 'Deprecated chrome option is ignored: useAutomationExtension\n',
 'timestamp': 1626926109381}

In [44]:
from selenium.common.exceptions import WebDriverException
try:
    print(driver.current_url )
except Exception as e :
    print('hello ex',e)

http://stockpage.10jqka.com.cn/000002/finance/#view


In [21]:
driver.back

<bound method WebDriver.back of <selenium.webdriver.chrome.webdriver.WebDriver (session="8d81ceeb416ec5053a8ba5bae004a93e")>>

In [23]:
print (driver.title)

WebDriverException: Message: chrome not reachable
  (Session info: chrome=92.0.4515.107)


In [22]:
'01,02,03'.split(',')

['01', '02', '03']

In [1]:
print('hello')

hell


In [7]:
import pandas as pd
#打开Excel时会报出Workbook corruption: seen[2] == 4错误。解决方案
#https://blog.csdn.net/sderaa/article/details/113059134
#进行百度，发现这个故障是说引用第三方库（xlrd）引起的。目前发现有两种解决方法：
'''
1.将Excel打开后再点击保存一下，就可以正常打开。
但这只是针对单个文件，对于多个文件非常麻烦。
2.采用网上的解决方法，修改xlrd库
这个可以从根本上解决。（参考如下文章：https://blog.csdn.net/chang1976272446/article/details/83588245）
从上面错误信息中找到compdoc.py程序地址，我的存储如下：
#File “D:\Python\lib\site-packages\xlrd\compdoc.py”, line 429, in _locate_stream

**把429这一行进行注释掉（我是把If条件全部注释掉，不然还会出现Print执行），最好先进行备份再修改。**
————————————————
版权声明：本文为CSDN博主「sderaa」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/sderaa/article/details/113059134

'''
df = pd.read_excel(r'D:/rep/600872_main_report.xls',sheet_name=0)

print(df.head())

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 1104-1105: truncated \xXX escape (<ipython-input-7-cc7127eac3a0>, line 18)

In [27]:
import locale
from locale import atof
locale.setlocale(locale.LC_NUMERIC, '')
#df.iloc[22:30,2:3].applymap(atof)


'Chinese (Simplified)_China.936'

In [7]:
import pandas as pd
import numpy as np


np.random.seed(24)
df = pd.DataFrame({'A': np.linspace(1, 10, 10)})
df = pd.concat([df, pd.DataFrame(np.random.randn(10, 4), columns=list('BCDE'))],
               axis=1)
df.iloc[3, 3] = np.nan
df.iloc[0, 2] = np.nan
df.style.format({"B": lambda x: "{}".format(wan_of_num(x*100000000000000000))}).bar(width=100,subset=pd.IndexSlice[3:5,['C','D','E']],color='lightpink')  #指定列级别自定义函数显示格式,指定特定行列


,A,B,C,D,E
0,1,13.292亿亿,nan,-0.31628,-0.99081
1,2,-10.708亿亿,-1.43871,0.564417,0.295722
2,3,-16.264亿亿,0.219565,0.678805,1.88927
3,4,9.615亿亿,0.104011,nan,0.850229
4,5,14.534亿亿,1.05774,0.165562,0.515018
5,6,-13.369亿亿,0.562861,1.39285,-0.063328
6,7,1.217亿亿,1.2076,-0.00204021,1.6278
7,8,3.545亿亿,1.03753,-0.385684,0.519818
8,9,16.866亿亿,-1.32596,1.42898,-2.08935
9,10,-1.298亿亿,0.631523,-0.586538,0.29072


In [2]:
########################################读取本地报表进行分析，实现了亿，万为单位显示数字
#   todo 补充关键指标 可持续增长率，内含增长率，增长模型，估值模型
#   相似多股同列，图表显示
#   自动查找本地文件，没有就自动下载后再加载
############################################################################################


import pandas as pd
import sys

def wan_of_num(num):  #实现以万，亿，万亿，亿亿为单位显示功能，需要修改得更加广谱和健壮些
    '''
    递归实现，精确为最大单位值 + 小数点后三位
    '''
    if isinstance(num,int) or  isinstance(num,float) :
        pass
    else :
        return num
        
    if '%' in str(num) :
        return num    

    if str(num).find('-') == 0:
        num =num*-1
        fuhao = True
    else :
        fuhao = False

    def strofsize(num, level):        

        if level >= 4:
            return num, level
        elif num >= 10000:
            num /= 10000
            level += 1
            return strofsize(num, level)
        else:
            return num, level
    units = ['', '万', '亿','万亿','亿亿']
    num, level = strofsize(num, 0)
    if level > len(units):
        level -= 1
    if fuhao : 
        return '-{}{}'.format(round(num, 3), units[level])
    else :
        return '{}{}'.format(round(num, 3), units[level])
 


df = pd.read_excel(r'D:/rep/000002_main_report.xls',sheet_name=0, index_col=[0])

#print(df.head())
#df.set_index(r'科目\时间')
df12 = df.T  #行列转换

####  以下两行解决表格显示宽度问题
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:1044px !important; text-align:left;}</style>"))

# 以下设置显示格式,选用链式设置方式   
#指定全局的自定义函数以万,亿为单位显示格式
df13 = df12.style.format( lambda x: "{}".format(wan_of_num(x))).\
    set_properties(subset=['销售毛利率'], **{'width': '30px'}).\
    bar(width=100,subset=['净利润(元)']).\
    set_properties(subset=['净利润同比增长率','扣非净利润同比增长率'], **{'width': '30px','text-align':'center','vertical-align':'middle'})                 
df13


科目\时间,净利润(元),净利润同比增长率,扣非净利润(元),扣非净利润同比增长率,营业总收入(元),营业总收入同比增长率,基本每股收益(元),每股净资产(元),每股资本公积金(元),每股未分配利润(元),每股经营现金流(元),销售净利率,销售毛利率,净资产收益率,净资产收益率-摊薄,营业周期(天),存货周转率(次),存货周转天数(天),应收账款周转天数(天),流动比率,速动比率,保守速动比率,产权比率,资产负债比率
2021-03-31,12.923亿,3.44%,11.572亿,13.76%,622.641亿,30.33%,0.111,19.43,1.59,8.58,1.45,4.03%,20.41%,0.57%,0.57%,1841.3,0.05,1836.73,4.57,1.18,0.35,0.34,6.88,81.37%
2020-12-31,415.155亿,6.80%,402.377亿,5.02%,4191.117亿,13.92%,3.62,19.32,1.6,8.47,4.58,14.15%,29.25%,20.13%,18.49%,1154.88,0.31,1152.74,2.14,1.17,0.35,0.34,6.77,81.28%
2020-09-30,198.628亿,8.89%,188.893亿,3.93%,2414.915亿,7.85%,1.741,17.45,1.61,8.9,2.87,12.45%,29.94%,10.13%,9.80%,1481.25,0.18,1478.64,2.61,1.14,0.34,0.34,7.46,83.32%
2020-06-30,125.075亿,5.62%,121.142亿,3.10%,1463.495亿,5.05%,1.11,16.84,1.66,8.27,1.95,12.73%,31.81%,6.44%,6.39%,1652.71,0.11,1649.86,2.85,1.15,0.34,0.33,7.74,83.87%
2020-03-31,12.494亿,11.49%,10.172亿,-9.86%,477.743亿,-1.24%,0.111,16.69,1.09,8.55,-0.26,5.09%,31.28%,0.66%,0.66%,2476.88,0.04,2472.53,4.35,1.15,0.33,0.32,7.77,84.24%
2019-12-31,388.721亿,15.10%,383.144亿,14.41%,3678.939亿,23.59%,3.47,16.64,1.1,8.44,4.04,14.99%,36.25%,22.47%,20.67%,1265.79,0.28,1264.04,1.75,1.13,0.33,0.33,7.76,84.36%
2019-09-30,182.407亿,30.43%,181.756亿,30.32%,2239.148亿,27.21%,1.631,14.8,1.06,8.68,0.15,12.77%,35.99%,11.16%,10.90%,1518.97,0.18,1516.85,2.12,1.13,0.31,0.3,8.33,85.06%
2019-06-30,118.418亿,29.79%,117.497亿,29.16%,1393.201亿,31.47%,1.06,14.25,1.08,8.12,0.78,13.84%,36.25%,7.26%,7.35%,1563.37,0.12,1561.14,2.23,1.12,0.33,0.33,8.35,85.26%
2019-03-31,11.206亿,25.23%,11.284亿,36.63%,483.746亿,56.93%,0.102,14.27,0.72,8.41,-2.42,6.63%,35.04%,0.72%,0.71%,2192.97,0.04,2189.78,3.19,1.16,0.36,0.35,8.35,84.77%
2018-12-31,337.727亿,20.39%,334.901亿,22.77%,2976.793亿,22.55%,3.06,14.11,0.73,8.31,3.05,16.55%,37.48%,23.24%,21.68%,1306.17,0.28,1304.35,1.83,1.15,0.4,0.4,8.3,84.59%


In [36]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:1044px !important; text-align:left;}</style>"))


In [ ]:
df.shape # (100, 6) 查看行数和列数
df.info() # 查看索引、数据类型和内存信息
df.describe() # 查看数值型列的汇总统计
df.dtypes # 查看各字段类型
df.axes # 显示数据行和列名
df.columns # 列名

In [35]:
import pandas as pd
import numpy as np
df = pd.read_csv(r'C:/Users/ccb/Documents/jams.csv',encoding='gb2312')
df.tail()
pd.pivot_table(df,index=[u'对手'])

,3分命中率,助攻,命中,得分,投篮命中率,投篮数,篮板
对手,,,,,,,
76人,0.33950,10.00,9.0,28.00,0.4405,20.5,3.5
勇士,0.44400,11.00,10.0,27.00,0.4350,23.0,6.0
国王,0.28600,9.00,8.0,27.00,0.3810,21.0,3.0
太阳,0.54500,7.00,12.0,48.00,0.5450,22.0,2.0
小牛,0.46200,7.00,10.0,29.00,0.5260,19.0,3.0
尼克斯,0.36900,9.50,10.5,34.00,0.4175,25.0,3.5
开拓者,0.57100,3.00,16.0,48.00,0.5520,29.0,8.0
掘金,0.14300,9.00,6.0,21.00,0.3750,16.0,8.0
步行者,0.29150,12.50,8.5,27.50,0.3965,21.5,6.5


In [38]:
df


,对手,胜负,主客场,命中,投篮数,投篮命中率,3分命中率,篮板,助攻,得分
0,勇士,胜,客,10,23,0.435,0.444,6,11,27
1,国王,胜,客,8,21,0.381,0.286,3,9,27
2,小牛,胜,主,10,19,0.526,0.462,3,7,29
3,灰熊,负,主,8,20,0.400,0.250,5,8,22
4,76人,胜,客,10,20,0.500,0.250,3,13,27
5,黄蜂,胜,客,8,18,0.444,0.400,10,11,27
6,灰熊,负,客,6,19,0.316,0.222,4,8,20
7,76人,负,主,8,21,0.381,0.429,4,7,29
8,尼克斯,胜,客,9,23,0.391,0.353,5,9,31
9,老鹰,胜,客,8,15,0.533,0.545,3,11,29


In [39]:
pd.pivot_table(df,index=[u'对手',u'主客场'])

3分命中率    助攻    命中    得分  投篮命中率   投篮数    篮板
对手  主客场                                             
76人 主    0.4290   7.0   8.0  29.0  0.381  21.0   4.0
    客    0.2500  13.0  10.0  27.0  0.500  20.0   3.0
勇士  客    0.4440  11.0  10.0  27.0  0.435  23.0   6.0
国王  客    0.2860   9.0   8.0  27.0  0.381  21.0   3.0
太阳  客    0.5450   7.0  12.0  48.0  0.545  22.0   2.0
小牛  主    0.4620   7.0  10.0  29.0  0.526  19.0   3.0
尼克斯 主    0.3850  10.0  12.0  37.0  0.444  27.0   2.0
    客    0.3530   9.0   9.0  31.0  0.391  23.0   5.0
开拓者 客    0.5710   3.0  16.0  48.0  0.552  29.0   8.0
掘金  主    0.1430   9.0   6.0  21.0  0.375  16.0   8.0
步行者 主    0.3330  10.0   8.0  29.0  0.364  22.0   8.0
    客    0.2500  15.0   9.0  26.0  0.429  21.0   5.0
湖人  客    0.4440   9.0  13.0  36.0  0.591  22.0   4.0
灰熊  主    0.3395   8.0   9.5  30.0  0.420  22.5   4.5
    客    0.3610   7.5   7.5  24.5  0.383  19.5   4.5
爵士  主    0.8750  13.0  19.0  56.0  0.760  25.0   2.0
    客    0.3330   3.0   8.0  29.0  0.421  19.0   5.0
猛龙  主    0.2730  11.0   8.0  38.0  0.320  25.0   6.0
篮网  主    0.6150   8.0  13.0  37.0  0.650  20.0  10.0
老鹰  客    0.5450  11.0   8.0  29.0  0.533  15.0   3.0
骑士  主    0.4290  13.0   8.0  35.0  0.381  21.0  11.0
鹈鹕  主    0.4000  17.0   8.0  26.0  0.500  16.0   1.0
黄蜂  客    0.4000  11.0   8.0  27.0  0.444  18.0  10.0

In [36]:
pd.pivot_table(df,index=[u'主客场',u'胜负'],values=[u'得分',u'助攻',u'篮板'],aggfunc=[np.sum,np.mean])

sum                mean                     
         助攻   得分  篮板         助攻         得分        篮板
主客场 胜负                                              
主   胜    95  308  49  10.555556  34.222222  5.444444
    负    26   89  15   8.666667  29.666667  5.000000
客   胜   108  384  59   9.000000  32.000000  4.916667
    负     8   20   4   8.000000  20.000000  4.000000

In [24]:
display(HTML('<h2>Hello, world!</h2>'))

In [ ]:
import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
from plotly import graph_objs as go
from datetime import datetime as dt
import json
import pandas as pd

df_fund_data = pd.read_csv('https://plot.ly/~jackp/17534.csv')
df_fund_data.head()

df_perf_summary = pd.read_csv('https://plot.ly/~jackp/17530.csv')
df_perf_summary.head()

df_cal_year = pd.read_csv('https://plot.ly/~jackp/17528.csv')
df_cal_year.head()

df_perf_pc = pd.read_csv('https://plot.ly/~jackp/17532.csv')

def make_dash_table( df ):
    ''' Return a dash definitio of an HTML table for a Pandas dataframe '''
    table = []
    for index, row in df.iterrows():
        html_row = []
        for i in range(len(row)):
            html_row.append( html.Td([ row[i] ]) )
        table.append( html.Tr( html_row ) )
    return table

modifed_perf_table = make_dash_table( df_perf_summary )

modifed_perf_table.insert( 
    0, html.Tr( [ 
            html.Td([]), 
            html.Td(['Cumulative'], colSpan=4, style=dict(textAlign="center")), 
            html.Td(['Annualised'], colSpan=4, style=dict(textAlign="center")) 
        ], style = dict( background='white', fontWeight=600 )
    ) 
)
    
df_fund_info = pd.read_csv('https://plot.ly/~jackp/17544.csv')
df_fund_characteristics = pd.read_csv('https://plot.ly/~jackp/17542.csv')
df_fund_facts = pd.read_csv('https://plot.ly/~jackp/17540.csv')
df_bond_allocation = pd.read_csv('https://plot.ly/~jackp/17538.csv')

dash.__version__

app = dash.Dash('GS Bond II Portfolio')


In [ ]:
app.layout = html.Div([        
        
    html.Div([ # page 1

        html.A([ 'Print PDF' ], 
           className="button no-print", 
           style=dict(position="absolute", top=-40, right=0)),     
                        
        html.Div([ # subpage 1

            # Row 1 (Header)

            html.Div([

                html.Div([      
                    html.H5('Goldman Sachs Strategic Absolute Return Bond II Portfolio'),
                    html.H6('A sub-fund of Goldman Sachs Funds, SICAV', style=dict(color='#7F90AC')),
                    ], className = "nine columns padded" ),

                html.Div([            
                    html.H1([html.Span('03', style=dict(opacity=0.5)), html.Span('17')]),
                    html.H6('Monthly Fund Update')
                ], className = "three columns gs-header gs-accent-header padded", style=dict(float='right') ),

            ], className = "row gs-header gs-text-header"),

            html.Br([]),

            # Row 2

            html.Div([     

                html.Div([
                    html.H6('Investor Profile', className = "gs-header gs-text-header padded"),

                    html.Strong('Investor objective'),
                    html.P('Capital appreciation and income.', className = 'blue-text'),

                    html.Strong('Position in your overall investment portfolio*'),
                    html.P('The fund can complement your portfolio.', className = 'blue-text'),

                    html.Strong('The fund is designed for:'),
                    html.P('The fund is designed for investors who are looking for a flexible \
                            global investment and sub-investment grade fixed income portfolio \
                            that has the ability to alter its exposure with an emphasis on interest \
                            rates, currencies and credit markets and that seeks to generate returns \
                            through different market conditions with a riskier investment strategy \
                            than GS Strategic Absolute Return Bond I Portfolio.', className = 'blue-text' ),

                ], className = "four columns" ),

                html.Div([
                    html.H6(["Performance (Indexed)"],
                            className = "gs-header gs-table-header padded"),
                    html.Iframe(src="https://plot.ly/~jackp/17553.embed?modebar=false&link=false&autosize=true", \
                            seamless="seamless", style=dict(border=0), width="100%", height="250")
                ], className = "eight columns" ),

            ], className = "row "),
                        
            # Row 2.5

            html.Div([     

                html.Div([
                    html.H6('Performance (%)', className = "gs-header gs-text-header padded"),
                    html.Table( make_dash_table( df_perf_pc ), className = 'tiny-header' )
                ], className = "four columns" ),

                html.Div([
                    html.P("This is an actively managed fund that is not designed to track its reference benchmark. \
                        Therefore the performance of the fund and the performance of its reference benchmark \
                        may diverge. In addition stated reference benchmark returns do not reflect any management \
                        or other charges to the fund, whereas stated returns of the fund do."),
                    html.Strong("Past performance does not guarantee future results, which may vary. \
                        The value of investments and the income derived from investments will fluctuate and \
                        can go down as well as up. A loss of capital may occur.")
                ], className = "eight columns" ),

            ], className = "row "),                        

            # Row 3

            html.Div([             

                html.Div([
                    html.H6('Fund Data', className = "gs-header gs-text-header padded"),
                    html.Table( make_dash_table( df_fund_data ) )
                ], className = "four columns" ),

                html.Div([
                    html.H6("Performance Summary (%)", className = "gs-header gs-table-header padded"),
                    html.Table( modifed_perf_table, className = "reversed" ),

                    html.H6("Calendar Year Performance (%)", className = "gs-header gs-table-header padded"),
                    html.Table( make_dash_table( df_cal_year ) )
                ], className = "eight columns" ),                      

            ], className = "row "),                

        ], className = "subpage" ),

    ], className = "page" ),  
             
    html.Div([ # page 2
                
        html.A([ 'Print PDF' ], 
           className="button no-print", 
           style=dict(position="absolute", top=-40, right=0)),                
                
        html.Div([ # subpage 2
                        
            # Row 1 (Header)
                        
            html.Div([

                html.Div([      
                    html.H5('Goldman Sachs Strategic  Absolute Return Bond II Portfolio'),
                    html.H6('A sub-fund of Goldman Sachs Funds, SICAV', style=dict(color='#7F90AC')),
                    ], className = "nine columns padded" ),

                html.Div([            
                    html.H1([html.Span('03', style=dict(opacity=0.5)), html.Span('17')]),
                    html.H6('Monthly Fund Update')
                ], className = "three columns gs-header gs-accent-header padded", style=dict(float='right') ),

            ], className = "row gs-header gs-text-header"),
                        
            # Row 2
                        
            html.Div([

                # Data tables on this page:
                # ---
                # df_fund_info = pd.read_csv('https://plot.ly/~jackp/17544/.csv')
                # df_fund_characteristics = pd.read_csv('https://plot.ly/~jackp/17542/.csv')
                # df_fund_facts = pd.read_csv('https://plot.ly/~jackp/17540/.csv')
                # df_bond_allocation = pd.read_csv('https://plot.ly/~jackp/17538/')                                
                  
                # Column 1
                                
                html.Div([      
                    html.H6('Financial Information', className = "gs-header gs-text-header padded"),
                    html.Table( make_dash_table( df_fund_info ) ),
                                        
                    html.H6('Fund Characteristics', className = "gs-header gs-text-header padded"),
                    html.Table( make_dash_table( df_fund_characteristics ) ),
                                        
                    html.H6('Fund Facts', className = "gs-header gs-text-header padded"),
                    html.Table( make_dash_table( df_fund_facts ) ),
                                        
                ], className = "four columns" ),
                                
                # Column 2

                html.Div([                                          
                    html.H6('Sector Allocation (%)', className = "gs-header gs-table-header padded"),
                    html.Iframe(src="https://plot.ly/~jackp/17560.embed?modebar=false&link=false&autosize=true", \
                        seamless="seamless", style=dict(border=0), width="100%", height="300"),
                                        
                    html.H6('Country Bond Allocation (%)', className = "gs-header gs-table-header padded"),
                    html.Table( make_dash_table( df_bond_allocation ) ),
                                        
                ], className = "four columns" ),
                                
                # Column 3
                                
                html.Div([            
                    html.H6('Top 10 Currency Weights (%)', className = "gs-header gs-table-header padded"),
                    html.Iframe(src="https://plot.ly/~jackp/17555.embed?modebar=false&link=false&autosize=true", \
                        seamless="seamless", style=dict(border=0), width="100%", height="300"),
                                        
                    html.H6('Credit Allocation (%)', className = "gs-header gs-table-header padded"),
                    html.Iframe(src="https://plot.ly/~jackp/17557.embed?modebar=false&link=false&autosize=true", \
                        seamless="seamless", style=dict(border=0), width="100%", height="300"),                                       
                                                                                
                ], className = "four columns" ),                                

            ], className = "row"),                        
                        
        ], className = "subpage" ),
                
    ], className = "page" ),
             
])


external_css = [ "https://cdnjs.cloudflare.com/ajax/libs/normalize/7.0.0/normalize.min.css",
        "https://cdnjs.cloudflare.com/ajax/libs/skeleton/2.0.4/skeleton.min.css",
        "//fonts.googleapis.com/css?family=Raleway:400,300,600",
        "https://codepen.io/plotly/pen/KmyPZr.css",
        "https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css"]

for css in external_css: 
    app.css.append_css({ "external_url": css })
    
external_js = [ "https://code.jquery.com/jquery-3.2.1.min.js",
        "https://codepen.io/plotly/pen/KmyPZr.js" ]
    
for js in external_js: 
    app.scripts.append_script({ "external_url": js })
    
app.server.run()
#* Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [ ]:
import plotly.express as px
df = px.data.iris()
#Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species','species_id'],dtype='object')
#   sepal_length sepal_width ...  species species_id
# 0       5.1     3.5 ...   setosa      1
# 1       4.9     3.0 ...   setosa      1
# 2       4.7     3.2 ...   setosa      1
# ..      ...     ... ...    ...     ...
# 149      5.9     3.0 ... virginica      3
# plotly.express.scatter(data_frame=None, x=None, y=None, 
# color=None, symbol=None, size=None,
# hover_name=None, hover_data=None, custom_data=None, text=None,
# facet_row=None, facet_col=None, facet_col_wrap=0, facet_row_spacing=None, facet_col_spacing=None,
# error_x=None, error_x_minus=None, error_y=None, error_y_minus=None,
# animation_frame=None, animation_group=None,
# category_orders=None, labels=None, orientation=None,
# color_discrete_sequence=None, color_discrete_map=None, color_continuous_scale=None, 
# range_color=None, color_continuous_midpoint=None,
# symbol_sequence=None, symbol_map=None, opacity=None, 
# size_max=None, marginal_x=None, marginal_y=None,
# trendline=None, trendline_color_override=None, 
# log_x=False, log_y=False, range_x=None, range_y=None,
# render_mode='auto', title=None, template=None, width=None, height=None)
# 以sepal_width，sepal_length制作标准散点图
fig = px.scatter(df, x="sepal_width", y="sepal_length")
fig.show()
  
  
#以鸢尾花类型-species作为不同颜色区分标志 color
fig = px.scatter(df, x="sepal_width", y="sepal_length", color="species")
fig.show()
  
#追加petal_length作为散点大小，变位气泡图 size
fig = px.scatter(df, x="sepal_width", y="sepal_length",
         color="species",size='petal_length')
fig.show()
  
#追加petal_width作为额外列，在悬停工具提示中显示为额外数据 hover_data
fig = px.scatter(df, x="sepal_width", y="sepal_length",
         color="species", size='petal_length',
         hover_data=['petal_width'])
fig.show()
  
#以鸢尾花类型-species区分散点的形状 symbol
fig = px.scatter(df, x="sepal_width", y="sepal_length",
         symbol="species" ,color="species", 
         size='petal_length',
         hover_data=['petal_width'])
fig.show()
  
#追加petal_width作为额外列，在悬停工具提示中以粗体显示。 hover_name
fig = px.scatter(df, x="sepal_width", y="sepal_length",
         symbol="species" ,color="species", 
         size='petal_length',
         hover_data=['petal_width'], hover_name="species")
fig.show()
  
#以鸢尾花类型编码-species_id作为散点的文本值 text
fig = px.scatter(df, x="sepal_width", y="sepal_length",
         symbol="species" ,color="species", 
         size='petal_length',
         hover_data=['petal_width'], hover_name="species",
         text="species_id")
fig.show()
  
#追加图表标题 title
fig = px.scatter(df, x="sepal_width", y="sepal_length",
         symbol="species" ,color="species", size='petal_length',
         hover_data=['petal_width'], hover_name="species",
         text="species_id",title="鸢尾花分类展示")
fig.show()
  
#以鸢尾花类型-species作为动画播放模式 animation_frame
fig = px.scatter(df, x="sepal_width", y="sepal_length",
         symbol="species" ,color="species", size='petal_length',
         hover_data=['petal_width'], hover_name="species",
         text="species_id",title="鸢尾花分类展示",
         animation_frame="species")
fig.show()
  
#固定X、Y最大值最小值范围range_x，range_y，防止动画播放时超出数值显示
fig = px.scatter(df, x="sepal_width", y="sepal_length",
         symbol="species" ,color="species", size='petal_length',
         hover_data=['petal_width'], hover_name="species",
         text="species_id",title="鸢尾花分类展示",
         animation_frame="species",range_x=[1.5,4.5],range_y=[4,8.5])
fig.show()
  
df = px.data.gapminder().query("country=='China'")
# Index(['country', 'continent', 'year', 'lifeExp', 'pop', 'gdpPercap', 'iso_alpha', 'iso_num'],dtype='object')
#   country continent year ...  gdpPercap iso_alpha iso_num
# 288  China   Asia 1952 ...  400.448611    CHN   156
# 289  China   Asia 1957 ...  575.987001    CHN   156
# 290  China   Asia 1962 ...  487.674018    CHN   156
# plotly.express.line(data_frame=None, x=None, y=None, 
# line_group=None, color=None, line_dash=None,
# hover_name=None, hover_data=None, custom_data=None, text=None,
# facet_row=None, facet_col=None, facet_col_wrap=0, 
# facet_row_spacing=None, facet_col_spacing=None,
# error_x=None, error_x_minus=None, error_y=None, error_y_minus=None,
# animation_frame=None, animation_group=None,
# category_orders=None, labels=None, orientation=None,
# color_discrete_sequence=None, color_discrete_map=None,
# line_dash_sequence=None, line_dash_map=None,
# log_x=False, log_y=False,
# range_x=None, range_y=None,
# line_shape=None, render_mode='auto', title=None, 
# template=None, width=None, height=None)
# 显示中国的人均寿命
fig = px.line(df, x="year", y="lifeExp", title='中国人均寿命')
fig.show()
  
# 以不同颜色显示亚洲各国的人均寿命
df = px.data.gapminder().query("continent == 'Asia'")
fig = px.line(df, x="year", y="lifeExp", color="country", 
       hover_name="country")
fig.show()
  
# line_group="country" 达到按国家去重的目的
df = px.data.gapminder().query("continent != 'Asia'") # remove Asia for visibility
fig = px.line(df, x="year", y="lifeExp", color="continent",
       line_group="country", hover_name="country")
fig.show()
  
# bar图
df = px.data.gapminder().query("country == 'China'")
fig = px.bar(df, x='year', y='lifeExp')
fig.show()
  
df = px.data.gapminder().query("continent == 'Asia'")
fig = px.bar(df, x='year', y='lifeExp',color="country" )
fig.show()
  
df = px.data.gapminder().query("country == 'China'")
fig = px.bar(df, x='year', y='pop',
       hover_data=['lifeExp', 'gdpPercap'], color='lifeExp',
       labels={'pop':'population of China'}, height=400)
fig.show()
  
fig = px.bar(df, x='year', y='pop',
       hover_data=['lifeExp', 'gdpPercap'], color='pop',
       labels={'pop':'population of China'}, height=400)
fig.show()
  
df = px.data.medals_long()
# #     nation  medal count
# # 0 South Korea  gold   24
# # 1    China  gold   10
# # 2    Canada  gold   9
# # 3 South Korea silver   13
# # 4    China silver   15
# # 5    Canada silver   12
# # 6 South Korea bronze   11
# # 7    China bronze   8
# # 8    Canada bronze   12
fig = px.bar(df, x="nation", y="count", color="medal", 
       title="Long-Form Input")
fig.show()
  
# 气泡图
df = px.data.gapminder()
# X轴以对数形式展现
fig = px.scatter(df.query("year==2007"), x="gdpPercap", y="lifeExp",
         size="pop",
         color="continent",hover_name="country", 
         log_x=True, size_max=60)
fig.show()
  
# X轴以标准形式展现
fig = px.scatter(df.query("year==2007"), x="gdpPercap", y="lifeExp",
         size="pop",
         color="continent",hover_name="country", 
         log_x=False, size_max=60)
fig.show()
  
# 饼状图
px.data.gapminder().query("year == 2007").groupby('continent').count()
#      country year lifeExp pop gdpPercap iso_alpha iso_num
# continent
# Africa     52  52    52  52     52     52    52
# Americas    25  25    25  25     25     25    25
# Asia      33  33    33  33     33     33    33
# Europe     30  30    30  30     30     30    30
# Oceania     2   2    2  2     2     2    2
df = px.data.gapminder().query("year == 2007").query("continent == 'Americas'")
fig = px.pie(df, values='pop', names='country',
       title='Population of European continent')
fig.show()
  
df.loc[df['pop'] < 10000000, 'country'] = 'Other countries'
fig = px.pie(df, values='pop', names='country', 
       title='Population of European continent',
       hover_name='country',labels='country')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()
  
df.loc[df['pop'] < 10000000, 'country'] = 'Other countries'
fig = px.pie(df, values='pop', names='country', 
       title='Population of European continent',
       hover_name='country',labels='country', 
       color_discrete_sequence=px.colors.sequential.Blues)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()
  
# 二维面积图
df = px.data.gapminder()
fig = px.area(df, x="year", y="pop", color="continent", 
       line_group="country")
fig.show()
  
fig = px.area(df, x="year", y="pop", color="continent", 
       line_group="country",
       color_discrete_sequence=px.colors.sequential.Blues)
fig.show()
  
df = px.data.gapminder().query("year == 2007")
fig = px.bar(df, x="pop", y="continent", orientation='h',
       hover_name='country',
       text='country',color='continent')
fig.show()
  
# 甘特图
import pandas as pd
df = pd.DataFrame([
  dict(Task="Job A", Start='2009-01-01', Finish='2009-02-28', 
     Completion_pct=50, Resource="Alex"),
  dict(Task="Job B", Start='2009-03-05', Finish='2009-04-15',
     Completion_pct=25, Resource="Alex"),
  dict(Task="Job C", Start='2009-02-20', Finish='2009-05-30', 
     Completion_pct=75, Resource="Max")
])
fig = px.timeline(df, x_start="Start", x_end="Finish", y="Task", 
         color="Completion_pct")
fig.update_yaxes(autorange="reversed")
fig.show()
  
fig = px.timeline(df, x_start="Start", x_end="Finish", y="Resource", 
         color="Resource")
fig.update_yaxes(autorange="reversed")
fig.show()
  
# 玫瑰环图
df = px.data.tips()
#   total_bill  tip   sex smoker  day  time size
# 0     16.99 1.01 Female   No  Sun Dinner   2
# 1     10.34 1.66  Male   No  Sun Dinner   3
# 2     21.01 3.50  Male   No  Sun Dinner   3
# 3     23.68 3.31  Male   No  Sun Dinner   2
# 4     24.59 3.61 Female   No  Sun Dinner   4
fig = px.sunburst(df, path=['day', 'time', 'sex'], 
         values='total_bill')
fig.show()
  
import numpy as np
df = px.data.gapminder().query("year == 2007")
fig = px.sunburst(df, path=['continent', 'country'], 
         values='pop',
         color='lifeExp', hover_data=['iso_alpha'],
         color_continuous_scale='RdBu',
         color_continuous_midpoint=np.average(df['lifeExp'], 
                            weights=df['pop']))
fig.show()
  
df = px.data.gapminder().query("year == 2007")
fig = px.sunburst(df, path=['continent', 'country'], 
         values='pop',
         color='pop', hover_data=['iso_alpha'],
         color_continuous_scale='RdBu')
fig.show()
  
# treemap图
import numpy as np
df = px.data.gapminder().query("year == 2007")
df["world"] = "world" # in order to have a single root node
fig = px.treemap(df, path=['world', 'continent', 'country'], 
         values='pop',
         color='lifeExp', hover_data=['iso_alpha'],
         color_continuous_scale='RdBu',
         color_continuous_midpoint=np.average(df['lifeExp'], 
                            weights=df['pop']))
fig.show()
  
fig = px.treemap(df, path=['world', 'continent', 'country'], values='pop',
         color='pop', hover_data=['iso_alpha'],
         color_continuous_scale='RdBu',
         color_continuous_midpoint=np.average(df['lifeExp'], 
                            weights=df['pop']))
fig.show()
  
fig = px.treemap(df, path=['world', 'continent', 'country'], values='pop',
         color='lifeExp', hover_data=['iso_alpha'],
         color_continuous_scale='RdBu')
fig.show()
  
fig = px.treemap(df, path=[ 'continent', 'country'], values='pop',
         color='lifeExp', hover_data=['iso_alpha'],
         color_continuous_scale='RdBu')
fig.show()
  
fig = px.treemap(df, path=[ 'country'], values='pop',
         color='lifeExp', hover_data=['iso_alpha'],
         color_continuous_scale='RdBu')
fig.show()
  
# 桑基图
tips = px.data.tips()
fig = px.parallel_categories(tips, color="size",
               color_continuous_scale=px.colors.sequential.Inferno)
fig.show()

In [ ]:
import pandas as pd
import numpy as np
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go

app = dash.Dash()
server = app.server
# 数据载入
df = pd.read_csv('https://raw.githubusercontent.com/ffzs/dataset/master/insurance.csv')
print(df)# app的layout 
app.layout = html.Div([
    html.Div([
        html.Label('Gender'),
        html.Div([
            dcc.Dropdown(   # 功能性组件， 设定id值作为标签关联callback函数中的标签
                id='gender',
                options=[{'label': i, 'value': i} for i in ['female', 'male']],
                value='female'),
        ]),
        html.Label('Color'),
        html.Div([
            dcc.Dropdown(
                id='color',
                options=[{'label': i, 'value': i} for i in ['region', 'smoker', 'children']],
                value='region'),
        ]),
    ], className="three columns"),  # 直接加入css的功能
    html.Div([
        dcc.Graph(id='scatter')    # 关联graph
    ], className="eight columns")
], className="page")

# 对callback函数进行设置， 与上面的对应， 将数据return回对应id的Graph
@app.callback(
    dash.dependencies.Output('scatter', 'figure'),
    [dash.dependencies.Input('gender','value'),
     dash.dependencies.Input('color','value')]
)
def update_scatter(value_gender, value_color):
    grouped = df.groupby('sex')  # 以性别分组
    data = grouped.get_group(value_gender)  # 获取选取的性别为变量
    # color_col = value_color   # 获取颜色分类的属性
    color_class = pd.Categorical(data[value_color])   # 将颜色分类数据明确化
#     c = [Spectral8[i] for i in color_class.codes]   # 获取颜色列表
    s = [(np.sqrt(i)+2) for i in data['age']]   # 将年龄数据开方用大小表示用以区别年龄大小
    trace = go.Scatter(
                x=data["bmi"],
                y=data["charges"],
                mode='markers',
                marker=dict(
                    size=s,
                    sizemode='diameter',
                    sizeref=0,
                    color=color_class.codes,
                    colorscale='Earth'
                ))

    layout = go.Layout(margin=dict(l=20, r=20, t=0, b=30))
    fig = go.Figure(data=[trace], layout=layout)
    return fig

# css的设置
external_css = ["https://cdnjs.cloudflare.com/ajax/libs/normalize/7.0.0/normalize.min.css",
                "https://cdnjs.cloudflare.com/ajax/libs/skeleton/2.0.4/skeleton.min.css",
                "https://codepen.io/ffzs/pen/mjjXGM.css",
                "https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css"]

for css in external_css:
    app.css.append_css({"external_url": css})


if __name__ == '__main__':
    app.run_server(host='0.0.0.0', debug=True)  # 运行app 设置host 默认是localhost

In [ ]:
# -*- coding: utf-8 -*-
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd

df = pd.read_csv(
    'https://gist.githubusercontent.com/chriddyp/'
    'c78bf172206ce24f77d6363a2d754b59/raw/'
    'c353e8ef842413cae56ae3920b8fd78468aa4cb2/'
    'usa-agricultural-exports-2011.csv')


def generate_talbe(dataframe, max_rows=10):
    return html.Table(
        # Header
        [html.Tr([html.Th(col) for col in dataframe.columns])] +

        # Body
        [html.Tr([
            html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
        ]) for i in range(min(len(dataframe), max_rows))]
    )


app = dash.Dash()

app.css.append_css(
    {"external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"})

app.layout = html.Div(children=[
    html.H4(children='2011年美国农业出口数据表'),
    generate_talbe(df)
])
if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
import dash_core_components as dcc
print(dcc.__version__)

In [30]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from jupyter_plotly_dash import JupyterDash
import time
print('jupyterDash is ok',time.localtime()[1:5])

jupyterDash is ok (8, 2, 20, 40)


In [50]:
df = pd.read_excel('d:\\rep\\000002_main_report.xls',header=1) #engine='openpyxl'
app = JupyterDash('input', height = 600)
app.add_external_link = False
app.layout = html.Div([
    html.H2('选项卡示例', style = dict(textAlign='center')),
    dcc.Tabs(
        id = 'tabs-example',
        value = 'tab-1',
        children = [dcc.Tab(label = '选项卡一', value = 'tab-1'),
                    dcc.Tab(label = '选项卡二', value = 'tab-2')]),
    html.Div(id = 'tabs-demo')
])

@app.callback(Output('tabs-demo', 'children'), [Input('tabs-example', 'value')])
def render_content(tab):
    if tab == 'tab-1':
        return html.Div([
            html.H3('内容一'),
            dcc.Graph(
                id='graph-1-tabs',
                figure = dict(data = [dict(x = [1, 2, 3,4], y = [3, 1, 2,6], type = 'bar')])
            )
        ])
    else:
        return html.Div([
            html.H3('内容二'),
            #html.H3(df.columns[0]),
            dcc.Graph(
                id='graph-2-tabs',
                figure = dict(data = [dict(x = df[df.columns[0]][6:10], y = df[df.columns[2]][6:10], type = 'bar', name = df.columns[2]),
                                      dict(x = df[df.columns[0]][6:10], y = df[df.columns[6]][6:10], type = 'bar', name = df.columns[6]),
                                      dict(x = df[df.columns[0]][6:10], y = df[df.columns[10]][6:10], type = 'bar', name = df.columns[10])])
            )
        ])

app

In [99]:
#todo 图表的设置,pandas的数据筛选,自动下载报表,板块对比分析,获取沪深列表

In [9]:
import pymssql
conn = pymssql.connect(host='ljl', port=59157 ,user='sa', password='123', database='JiXiao', charset='utf8') #, charset='cp936' gbk

df = pd.read_sql("select * from test1",con=conn)

conn.close()
df
#df.loc[1].at[0]

,rowid,姓名,金额,备注,colsrc
0,1,张三,1.0,五,None
1,2,李四,2.0,感觉,None


In [4]:
#df = pd.read_excel('d:\\rep\\hsA.xls')
df = pd.read_excel('https://www.gairuo.com/file/data/team.xlsx')

In [3]:
df = pd.read_excel('https://www.gairuo.com/file/data/team.xlsx')

df


,name,team,Q1,Q2,Q3,Q4
0,Liver,E,89,21,24,64
1,Arry,C,36,37,37,57
2,Ack,A,57,60,18,84
3,Eorge,C,93,96,71,78
4,Oah,D,65,49,61,86
...,...,...,...,...,...,...
95,Gabriel,C,48,59,87,74
96,Austin7,C,21,31,30,43
97,Lincoln4,C,98,93,1,20
98,Eli,E,11,74,58,91


In [12]:
df.set_index('name', inplace=True)

In [13]:
df.loc['Ben', 'Q1':'Q4']

Q1    21
Q2    43
Q3    41
Q4    74
Name: Ben, dtype: object

In [14]:
df.loc[:, ['team', 'Q1']]

,team,Q1
name,,
Liver,E,89
Arry,C,36
Ack,A,57
Eorge,C,93
Oah,D,65
...,...,...
Gabriel,C,48
Austin7,C,21
Lincoln4,C,98


In [15]:
df[df.index == 'Liver']

,team,Q1,Q2,Q3,Q4
name,,,,,
Liver,E,89,21,24,64


In [18]:
df.iloc[4:10,2:5]

,Q2,Q3,Q4
name,,,
Oah,49,61,86
Harlie,13,87,43
Acob,95,94,8
Lfie,10,99,37
Reddie,93,57,72
Oscar,9,26,67


In [19]:
df.loc['Eorge':'Alexander', 'team':'Q4']

,team,Q1,Q2,Q3,Q4
name,,,,,
Eorge,C,93,96,71,78
Oah,D,65,49,61,86
Harlie,C,24,13,87,43
Acob,B,61,95,94,8
Lfie,A,9,10,99,37
Reddie,D,64,93,57,72
Oscar,A,77,9,26,67
Leo,B,17,4,33,79
Logan,B,9,89,35,65


In [20]:
df[df.team == 'C'] 

,team,Q1,Q2,Q3,Q4
name,,,,,
Arry,C,36,37,37,57
Eorge,C,93,96,71,78
Harlie,C,24,13,87,43
Archie,C,83,89,59,68
Theo,C,51,86,87,27
William,C,80,68,3,26
Daniel,C,50,50,72,61
Alexander,C,91,76,26,79
Adam,C,90,32,47,39


In [33]:
df.groupby('team').sum()

,Q1,Q2,Q3,Q4
team,,,,
A,1066,639,875,783
B,975,1218,1202,1136
C,1056,1194,1068,1127
D,860,1191,1241,1199
E,963,1013,881,1033


In [34]:
df.groupby('team').sum().stack()

team    
A     Q1    1066
      Q2     639
      Q3     875
      Q4     783
B     Q1     975
      Q2    1218
      Q3    1202
      Q4    1136
C     Q1    1056
      Q2    1194
      Q3    1068
      Q4    1127
D     Q1     860
      Q2    1191
      Q3    1241
      Q4    1199
E     Q1     963
      Q2    1013
      Q3     881
      Q4    1033
dtype: int64

In [51]:
df.at['Arry', 'Q1']

36

In [33]:
df_li = pd.read_excel('d:\\rep\\hsA.xls',converters={'name':str,'code':str}) #index_col = 'code'
print(df_li.at[2,'name'],df_li[df_li.code == '000002' ].iat[0,0] )
#df_li#.iat[0, 1] 

中环海陆 万 科Ａ


In [92]:
df.get(3, 0)

0

In [34]:
df_li.iloc[np.r_[:5, 10, 15:30:2, 70:75]]

,name,code,brie
0,金百泽,301041,NaN
1,久祺股份,300994,NaN
2,中环海陆,301040,NaN
3,深水规院,301038,NaN
4,保立佳,301037,NaN
10,仕净科技,301030,NaN
15,中旗新材,001212,NaN
17,容知日新,688768,NaN
19,金迪克,688670,NaN
21,正元地信,688509,NaN


In [51]:
df_tmp2 = df.iloc[:, np.r_[0, 2:21:4]]
df_tmp2 

,科目\时间,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31
0,净利润(元),41515544900,38872086900,33772651700,28051814900,21022606300
1,净利润同比增长率,6.80%,15.10%,20.39%,33.44%,16.02%
2,扣非净利润(元),40237711100,38314387500,33490078400,27279538700,20929278900
3,扣非净利润同比增长率,5.02%,14.41%,22.77%,30.34%,18.81%
4,营业总收入(元),4.19112e+11,3.67894e+11,2.97679e+11,2.42897e+11,2.40477e+11
5,营业总收入同比增长率,13.92%,23.59%,22.55%,1.01%,22.98%
6,基本每股收益(元),3.62,3.47,3.06,2.54,1.9
7,每股净资产(元),19.32,16.64,14.11,12.02,10.28
8,每股资本公积金(元),1.6,1.1,0.73,0.75,0.75
9,每股未分配利润(元),8.47,8.44,8.31,6.99,5.54


In [54]:
code = '000002'
df_tmp['code'] = df_li[df_li.code == code ].iat[0,0]
df_tmp

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,科目\时间,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31,code
0,净利润(元),41515544900,38872086900,33772651700,28051814900,21022606300,万 科Ａ
1,净利润同比增长率,6.80%,15.10%,20.39%,33.44%,16.02%,万 科Ａ
2,扣非净利润(元),40237711100,38314387500,33490078400,27279538700,20929278900,万 科Ａ
3,扣非净利润同比增长率,5.02%,14.41%,22.77%,30.34%,18.81%,万 科Ａ
4,营业总收入(元),4.19112e+11,3.67894e+11,2.97679e+11,2.42897e+11,2.40477e+11,万 科Ａ
5,营业总收入同比增长率,13.92%,23.59%,22.55%,1.01%,22.98%,万 科Ａ
6,基本每股收益(元),3.62,3.47,3.06,2.54,1.9,万 科Ａ
7,每股净资产(元),19.32,16.64,14.11,12.02,10.28,万 科Ａ
8,每股资本公积金(元),1.6,1.1,0.73,0.75,0.75,万 科Ａ
9,每股未分配利润(元),8.47,8.44,8.31,6.99,5.54,万 科Ａ


In [68]:
frames = [df_tmp, df_tmp1]
df_h = pd.concat(frames)
df_h[df_h.index == 13] 

,科目\时间,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31,code
13,净资产收益率,20.13%,22.47%,23.24%,22.80%,19.68%,万 科Ａ
13,净资产收益率,17.10%,21.01%,16.63%,16.32%,15.53%,保利地产
